In [1]:
import pandas as pd
import pyodbc

# Define file path
file_path = "C:/Users/1/Desktop/Final Project/Characterizing-Parliament-Members-using-LLM/db_csv/KNS_members_description_3.xlsx"

# Read the Excel file
df = pd.read_excel(file_path, usecols=["kns_name", "description", "kns_name_hebrew", "knesset_seat"])

# Define database connection parameters
server = "DESKTOP-9LFS960\SQLEXPRESS"  # Change to your server name
database = "KNS_Description"  # Change to your database name

dsn = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes'

# Establish connection
conn = pyodbc.connect(dsn)
cursor = conn.cursor()


In [2]:
df

,kns_name,description,kns_name_hebrew,knesset_seat
0,Amichai Eliyahu,Amichai Eliyahu is a National Religious and co...,אליהו עמיחי,25
1,Ofir Akunis,Ofir Akunis is a veteran Likud politician know...,אופיר אקוניס,"18,19,20,21,22,23,24,25"
2,Haim Biton,"Haim Biton is a key figure in the Shas party, ...",חיים ביטון,25
3,Yoav Ben Tzur,Yoav Ben-Tzur is a senior Shas party member kn...,יואב בן צור,"19,20,21,22,23,24,25"
4,Avraham Betzalel,Avraham Betzalel is a Shas MK who focuses on h...,אברהם בצלאל,"23,24,25"
5,Chaim Katz,Chaim Katz is a senior Likud politician with a...,חיים כץ,"15,16,17,18,19,20,21,22,23,24,25"
6,Uri Maklev,Uri Maklev is a veteran Knesset member from th...,אורי מקלב,"17,18,19,20,21,22,23,24,25"
7,Yaakov Margi,Yaakov Margi is a senior Shas politician known...,יעקב מרגי,"19,20,21,22,23,24,25"
8,Yoel Razvozov,Yoel Razvozov is a former Olympic judoka turne...,יואל רזבוזוב,"17,18,19,20,21,22,23,24,25"


In [3]:
# Insert data into kns_history table
for index, row in df.iterrows():
    cursor.execute("""
        INSERT INTO KNS_Description.dbo.kns_members (kns_name, description, kns_name_hebrew, knesset_seat)
        VALUES (?, ?, ?, ?)""",
        row["kns_name"], row["description"], row["kns_name_hebrew"], row["knesset_seat"]
    )

# Commit and close connection
conn.commit()
cursor.close()
conn.close()

print("Data inserted successfully!")

Data inserted successfully!


In [21]:
def get_knesset_numbers(name):
    conn = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};'
        'SERVER=DESKTOP-9LFS960\SQLEXPRESS;'
        'DATABASE=KNS_Description;'
        'Trusted_Connection=yes'
    )
    cursor = conn.cursor()
    query = "SELECT knesset_seat FROM kns_members WHERE kns_name = ?"
    cursor.execute(query, name)
    seats = [row[0] for row in cursor.fetchall()]
    result = [int(x) for x in seats[0].split(',')]
    result.insert(0, "All")
    conn.close()
    return result

In [22]:
get_knesset_numbers("Miri Regev")  # Replace with actual name to test

['All', 18, 19, 20, 21, 22, 23, 24, 25]

In [ ]:
def get_kns_names_and_hebrew():
  conn = pyodbc.connect(
      'DRIVER={ODBC Driver 17 for SQL Server};'
      'SERVER=DESKTOP-9LFS960\SQLEXPRESS;'
      'DATABASE=KNS_Description;'
      'Trusted_Connection=yes'
  )
  cursor = conn.cursor()
  query = "SELECT kns_name, kns_name_hebrew FROM kns_members"
  cursor.execute(query)
  names = []
  names_hebrew = []
  for row in cursor.fetchall():
    names.append(row[0])
    names_hebrew.append(row[1])
  conn.close()
  return names, names_hebrew

In [ ]:
get_kns_names_and_hebrew()